In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   


# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [47]:
s3 = boto3.client('s3')
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = sess.default_bucket()  # this could also be a hard-coded bucket name
print('Using bucket ' + bucket)

Using bucket sagemaker-us-west-2-929878552275


In [2]:
bucket_name = 'drugs-public' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [3]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='drugs-public'
data_key = 'df_ligand_protein.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df_3d=pd.read_csv(data_location)

In [4]:
data_key = 'morgan_matrix_generated_ligandprotein.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

morgan_matrix_df=pd.read_csv(data_location)

In [5]:
df_3d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52283 entries, 0 to 52282
Data columns (total 3 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   bindingdb_target_chain__sequence  52283 non-null  object 
 1   ligand_smiles                     52283 non-null  object 
 2   bindingaffinity                   52283 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.2+ MB


In [6]:
df_smiles=df_3d['ligand_smiles']
df_protein=df_3d['bindingdb_target_chain__sequence']
df_ba=df_3d['bindingaffinity']

In [7]:
dindex=[1460,4119,4120,4121,4122,4123,4124,4193,4194,4195]

In [8]:
df_protein.drop(df_protein.index[dindex], axis=0, inplace=True)
df_ba.drop(df_ba.index[dindex], axis=0, inplace=True)

In [9]:
from sklearn.model_selection import train_test_split
import sklearn.decomposition
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
def ligand_features(morgan_matrix_df,df_ba,test_size=0.2):
    print('Generating ligand features...')
    LX_train,LX_test,Ly_train,Ly_test=train_test_split(morgan_matrix_df,df_ba,test_size=0.2,random_state=42,shuffle=False)
    #Dimensionality reduction
    pca=PCA(.90)
    pca.fit(LX_train)
    print('Ligand features after PCA:',pca.n_components_)
    LX_train = pca.transform(LX_train)
    LX_test = pca.transform(LX_test)
    print('Ligand features complete!')
    return LX_train,LX_test,Ly_train,Ly_test

In [11]:
def protein_features(df_protein, df_ba, test_size = 0.2):
    print('Generating protein features...')   
    PX_train,PX_test,y_train,y_test = train_test_split(df_protein, df_ba, test_size = 0.2, random_state = 42,shuffle=False)

    # Create a Count Vectorizer to gather the unique elements in sequence
    vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (4,4))

    # Fit and Transform CountVectorizer
    vect.fit(PX_train)
    PX_train = vect.transform(PX_train)
    PX_test = vect.transform(PX_test)
    
    #Dimensionality reduction
    svd = sklearn.decomposition.TruncatedSVD(n_components=50)
    PX_train = svd.fit_transform(PX_train)
    PX_test = svd.transform(PX_test)

    #print('Protein features':vect.get_feature_names()[-20:])
    print('Protein features complete!')
    return PX_train,PX_test,y_train,y_test

In [12]:
test_size=0.2
LX_train,LX_test,Ly_train,Ly_test=ligand_features(morgan_matrix_df,df_ba,test_size)
PX_train,PX_test,Py_train,Py_test=protein_features(df_protein,df_ba,test_size)

Generating ligand features...
Ligand features after PCA: 191
Ligand features complete!
Generating protein features...
Protein features complete!


In [13]:
print(LX_train.shape),print(PX_train.shape)
print(LX_test.shape),print(PX_test.shape)

(41818, 191)
(41818, 50)
(10455, 191)
(10455, 50)


(None, None)

In [14]:
PX_train_df = pd.DataFrame(PX_train)
LX_train_df = pd.DataFrame(LX_train)

PX_test_df = pd.DataFrame(PX_test)
LX_test_df = pd.DataFrame(LX_test)

In [15]:
PX_train_df.columns = PX_train_df.columns.map(lambda x: str(x) + '_b')
PX_test_df.columns = PX_test_df.columns.map(lambda x: str(x) + '_b')

Merge protein and ligand features

In [16]:
df_total_train=PX_train_df.join(LX_train_df)
df_total_test=PX_test_df.join(LX_test_df)

y_train=pd.DataFrame(Py_train)
y_test=pd.DataFrame(Py_test)

In [17]:
df_total_train.reset_index(inplace=True,drop=True)
df_total_test.reset_index(inplace=True,drop=True)

y_train.reset_index(inplace=True,drop=True)
y_test.reset_index(inplace=True,drop=True)

In [18]:
y_test

,bindingaffinity
0,7.309804
1,7.366532
2,5.000000
3,5.000000
4,5.000000
...,...
10450,8.568636
10451,9.397940
10452,10.346787
10453,7.769551


In [19]:
#https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_randomforest/Sklearn_on_SageMaker_end2end.ipynb

trainX = pd.DataFrame(df_total_train)
trainX['target'] = y_train

testX = pd.DataFrame(df_total_test)
testX['target'] = y_test

In [20]:
testX.head()

,0_b,1_b,2_b,3_b,4_b,5_b,6_b,7_b,8_b,9_b,...,182,183,184,185,186,187,188,189,190,target
0,1.208051,0.604058,0.473296,0.289918,0.790123,0.400958,-1.836151,-0.52466,-0.258139,-0.073828,...,0.020933,0.000002,0.049205,0.018360,0.051950,-0.001727,-0.011288,0.032895,0.005646,7.309804
1,1.208051,0.604058,0.473296,0.289918,0.790123,0.400958,-1.836151,-0.52466,-0.258139,-0.073828,...,0.029299,0.006279,-0.006265,0.046739,0.004290,0.010059,0.008890,-0.005812,0.014569,7.366532
2,1.208051,0.604058,0.473296,0.289918,0.790123,0.400958,-1.836151,-0.52466,-0.258139,-0.073828,...,0.010844,-0.034868,-0.008864,-0.022088,0.019633,0.027510,-0.009552,0.036593,0.024887,5.000000
3,1.208051,0.604058,0.473296,0.289918,0.790123,0.400958,-1.836151,-0.52466,-0.258139,-0.073828,...,-0.007236,-0.043480,-0.020554,0.016175,0.042715,0.004216,0.017176,0.017251,0.010325,5.000000
4,2.399342,1.521650,1.386567,1.351741,1.845702,0.787432,-4.008339,-0.86938,-0.514733,0.158044,...,-0.004017,0.002042,0.019585,-0.029880,-0.039547,-0.024458,-0.017620,0.012704,-0.004906,5.000000


In [21]:
trainX.to_csv('train.csv')
testX.to_csv('validation.csv')

In [22]:
#testX.isnull().sum(axis = 1)

In [23]:
sess = sagemaker.Session()
trainpath = sess.upload_data(
    path='train.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

testpath = sess.upload_data(
    path='validation.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

In [25]:
%%writefile rf_script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf



if __name__ =='__main__':

    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument('--n-estimators', type=int, default=10)
    parser.add_argument('--min-samples-leaf', type=int, default=3)

    # Data, model, and output directories
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='train.csv')
    parser.add_argument('--test-file', type=str, default='validation.csv')
#    parser.add_argument('--features', type=str)  # in this script we ask user to explicitly name features
#    parser.add_argument('--target', type=str) # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print('reading data')
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print('building training and testing datasets')
    X_train=train_df.drop('target',axis=1)
    y_train=train_df['target']
    
    X_test=test_df.drop('target',axis=1)
    y_test=test_df['target']
    
#    X_train = train_df[args.features.split()]
#    X_test = test_df[args.features.split()]
#    y_train = train_df[args.target]
#    y_test = test_df[args.target]

    # train
    print('training model')
    model = RandomForestRegressor(n_estimators=args.n_estimators,min_samples_leaf=args.min_samples_leaf,n_jobs=-1)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)

#    rf_model = Pipeline([('scaler', StandardScaler()),('rf_model',RandomForestRegressor(random_state=1))])
#    params_rf = {'rf_model__n_estimators': [100, 200, 300],'rf_model__max_features': ['log2', 'auto', 'sqrt'],'rf_model__min_samples_leaf': [10, 30, 50]}
#    grid_rf=GridSearchCV(estimator=rf_model, param_grid=params_rf, scoring='neg_mean_absolute_error', cv=3, verbose=1, n_jobs=-1)

    #Fit and predict
#    grid_rf.fit(X_train, y_train)
#    test_pred = grid_rf.predict(X_test)
    
    mae_test = MAE(y_test, test_pred)
    mse_test = MSE(y_test, test_pred)   
    rmse_test=np.sqrt(mse_test)
    print('MAE:', mae_test)
    print('RMSE:',rmse_test)
    
    # print abs error
#    print('validating model')
#    abs_err = np.abs(model.predict(X_test) - y_test)
#    print(abs_err)

    # print couple perf metrics
 #   for q in [10, 50, 90]:
 #       print('AE-at-' + str(q) + 'th-percentile: '
 #             + str(np.percentile(a=abs_err, q=q)))
        
    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print('model persisted at ' + path)
    print(args.min_samples_leaf)

Overwriting rf_script.py


In [26]:
#Local training
#! python rf_script.py --n-estimators 100 --min-samples-leaf 2 --model-dir ./ --train ./ --test ./ 

In [36]:
#Python SDK training

from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = '0.23-1'

sklearn_estimator = SKLearn(
    entry_point='rf_script.py',
    role = get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.c5.xlarge',
    framework_version=FRAMEWORK_VERSION,
    base_job_name='rf-scikit',
 #   metric_definitions=[
 #       {'Name': 'median-AE',
 #        'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters = {'n-estimators': 100,
                       'min-samples-leaf': 3})
 #                      'features': 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT',
 #                      'target': 'target'})    

In [37]:
# launch training job, with asynchronous call
sklearn_estimator.fit({'train':trainpath, 'test': testpath}, wait=False)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [ ]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = sklearn_estimator.deploy(instance_type='ml.c5.xlarge',initial_instance_count=1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------

<b>Hyperparameter Tuning</b>

In [38]:
from sagemaker.tuner import IntegerParameter

# Define exploration boundaries
hyperparameter_ranges = {
    'n-estimators': IntegerParameter(20, 100),
    'min-samples-leaf': IntegerParameter(2, 6)}

# create Optimizer
Optimizer = sagemaker.tuner.HyperparameterTuner(
    estimator=sklearn_estimator,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name='RF-tuner',
    objective_type='Minimize',
    objective_metric_name='median-AE',
    metric_definitions=[
        {'Name': 'median-AE',
         'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}],  # extract tracked metric from logs with regexp 
    max_jobs=20,
    max_parallel_jobs=2)

In [39]:
Optimizer.fit({'train': trainpath, 'test': testpath})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [44]:
import datetime
import time
# get tuner results in a df
results = Optimizer.analytics().dataframe()
while results.empty:
    time.sleep(1)
    results = Optimizer.analytics().dataframe()
results.head()

,min-samples-leaf,n-estimators,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,3.0,100.0,RF-tuner-201015-0255-004-98812e35,InProgress,None,2020-10-15 03:02:34+00:00,NaT,NaN
1,2.0,99.0,RF-tuner-201015-0255-003-7797d702,InProgress,None,2020-10-15 03:02:18+00:00,NaT,NaN
2,2.0,59.0,RF-tuner-201015-0255-002-9e91c0af,Completed,None,2020-10-15 02:57:11+00:00,2020-10-15 03:00:08+00:00,177.0
3,4.0,74.0,RF-tuner-201015-0255-001-06273d53,Completed,None,2020-10-15 02:57:02+00:00,2020-10-15 03:00:18+00:00,196.0


In [45]:
sklearn_estimator.latest_training_job.wait(logs='None')
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)


2020-10-15 02:59:12 Starting - Preparing the instances for training
2020-10-15 02:59:12 Downloading - Downloading input data
2020-10-15 02:59:12 Training - Training image download completed. Training in progress.
2020-10-15 02:59:12 Uploading - Uploading generated training model
2020-10-15 02:59:12 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-west-2-929878552275/rf-scikit-2020-10-15-02-52-50-981/output/model.tar.gz


In [35]:
sklearn_estimator.latest_training_job.wait(logs='None')
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)


2020-10-15 02:47:39 Starting - Preparing the instances for training
2020-10-15 02:47:39 Downloading - Downloading input data
2020-10-15 02:47:39 Training - Training image download completed. Training in progress.
2020-10-15 02:47:39 Uploading - Uploading generated training model
2020-10-15 02:47:39 Completed - Training job completed


NameError: name 'sm_boto3' is not defined

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point='rf_script.py',
    framework_version=FRAMEWORK_VERSION)

In [ ]:
predictor = model.deploy(
    instance_type='ml.c5.xlarge',
    initial_instance_count=1)

Hyper-parameter tuning

In [ ]:
from sagemaker.tuner import IntegerParameter

# Define exploration boundaries
hyperparameter_ranges = {
    'n-estimators': IntegerParameter(20, 100),
    'min-samples-leaf': IntegerParameter(2, 6)}

# create Optimizer
Optimizer = sagemaker.tuner.HyperparameterTuner(
    estimator=sklearn_estimator,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name='RF-tuner',
    objective_type='Minimize',
    objective_metric_name='median-AE',
    metric_definitions=[
        {'Name': 'median-AE',
         'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}],  # extract tracked metric from logs with regexp 
    max_jobs=20,
    max_parallel_jobs=2)

In [21]:
train_data=y_train.join(df_total_train)
test_data=y_test.join(df_total_test)

In [22]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10455 entries, 0 to 10454
Columns: 242 entries, bindingaffinity to 190
dtypes: float64(242)
memory usage: 19.3 MB


In [23]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')    

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [24]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, train_instance_count=1, train_instance_type='ml.t2.large',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='reg:linear',num_round=100)


Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [39]:
xgb.fit({'train': s3_input_train})

2020-10-12 02:43:40 Starting - Starting the training job...
2020-10-12 02:43:42 Starting - Launching requested ML instances......
2020-10-12 02:44:45 Starting - Preparing the instances for training...
2020-10-12 02:45:35 Downloading - Downloading input data...
2020-10-12 02:46:11 Training - Downloading the training image...
2020-10-12 02:46:31 Training - Training image download completed. Training in progress.Arguments: train
[2020-10-12:02:46:31:INFO] Running standalone xgboost training.
[2020-10-12:02:46:31:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-10-12:02:46:31:INFO] File size need to be processed in the node: 195.11mb. Available memory size in the node: 8474.05mb
[2020-10-12:02:46:31:INFO] Determined delimiter of CSV input is ','
[02:46:31] S3DistributionType set as FullyReplicated
[02:46:32] 41818x240 matrix with 10036320 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[02:46:32] src/tree/updater_prune.cc:74: tree pruning en

In [40]:
#Deploy model
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------!

In [41]:
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

ValueError: ('Unable to handle input format: ', <class 'int'>)

In [ ]:
predictions = []
for array in modelData:
    result = linear_predictor.predict(array)
    predictions += [r['score'] for r in result['predictions']]
predictions = np.array(predictions)
# Push into our pandas dataframe
data['Predicted'] = predictions.astype(int))